# 🚀 XPLIA - Quick Start Guide

Bienvenue dans ce notebook interactif pour découvrir XPLIA!

## Objectifs
- Installer et importer XPLIA
- Créer votre premier explainer
- Générer des explications
- Visualiser les résultats

## 1. Installation

Si XPLIA n'est pas encore installé:

In [ ]:
# Décommenter pour installer
# !pip install xplia[xai,viz]

## 2. Imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Imports XPLIA
from xplia.core import create_explainer, ModelFactory, ExplainerFactory
from xplia.utils import Timer, measure_performance

print("✓ Imports réussis!")

## 3. Charger et Préparer les Données

In [ ]:
# Charger le dataset Iris
iris = load_iris()
X, y = iris.data, iris.target
feature_names = iris.feature_names
target_names = iris.target_names

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Dataset: {X.shape[0]} samples, {X.shape[1]} features")
print(f"Classes: {target_names}")
print(f"Features: {feature_names}")

## 4. Entraîner un Modèle

In [ ]:
# Entraîner avec mesure de performance
with measure_performance("Training", track_memory=True) as metrics:
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Score
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)

print(f"\n📊 Résultats:")
print(f"  Train accuracy: {train_score:.3f}")
print(f"  Test accuracy:  {test_score:.3f}")
print(f"\n⏱️  Performance:")
print(f"  Temps: {metrics['elapsed_time']:.3f}s")
print(f"  Mémoire: {metrics['memory_used']:.2f}MB")

## 5. Détecter le Type de Modèle

In [ ]:
# Détection automatique
model_type = ModelFactory.detect_model_type(model)
print(f"Type de modèle détecté: {model_type}")

# Méthode recommandée
recommended_method = ExplainerFactory.get_recommended_method(model)
print(f"Méthode d'explicabilité recommandée: {recommended_method}")

## 6. Créer un Explainer

In [ ]:
# Créer l'explainer avec la méthode unifiée
explainer = create_explainer(model, method='unified')
print(f"✓ Explainer créé: {type(explainer).__name__}")

## 7. Générer des Explications

In [ ]:
# Expliquer les 5 premières instances de test
with Timer("Explication", verbose=True):
    explanation = explainer.explain(X_test[:5])

print(f"\n✓ Explications générées pour {len(X_test[:5])} instances")

## 8. Analyser les Résultats

In [ ]:
# Feature importances
print("\n📊 Top 3 Features Importantes:")
print("=" * 50)
for i, fi in enumerate(explanation.feature_importances[:3], 1):
    print(f"{i}. {fi.feature_name:20s} : {fi.importance_value:.4f}")

# Créer un DataFrame pour visualisation
importance_df = pd.DataFrame([
    {
        'Feature': fi.feature_name,
        'Importance': fi.importance_value,
        'Rank': fi.importance_rank
    }
    for fi in explanation.feature_importances
])

print("\n📈 Toutes les features:")
print(importance_df.to_string(index=False))

## 9. Visualisation (si matplotlib disponible)

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    # Bar plot des importances
    plt.figure(figsize=(10, 6))
    plt.barh(importance_df['Feature'], importance_df['Importance'])
    plt.xlabel('Importance')
    plt.title('Feature Importances - XPLIA')
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("⚠️  Matplotlib non disponible pour la visualisation")

## 10. Prédictions et Explications Individuelles

In [ ]:
# Prendre une instance spécifique
instance_idx = 0
instance = X_test[instance_idx:instance_idx+1]
true_label = y_test[instance_idx]
pred_label = model.predict(instance)[0]
pred_proba = model.predict_proba(instance)[0]

print(f"\n🔍 Analyse de l'instance #{instance_idx}:")
print("=" * 50)
print(f"Vraie classe: {target_names[true_label]}")
print(f"Prédiction:   {target_names[pred_label]}")
print(f"\nProbabilités:")
for i, (name, proba) in enumerate(zip(target_names, pred_proba)):
    print(f"  {name:15s}: {proba:.3f} {'✓' if i == pred_label else ''}")

print(f"\nValeurs des features:")
for fname, value in zip(feature_names, instance[0]):
    print(f"  {fname:30s}: {value:.2f}")

## 🎉 Félicitations!

Vous avez complété le quick start de XPLIA!

### Prochaines étapes:
1. Explorer les autres notebooks
2. Essayer différentes méthodes d'explicabilité
3. Tester sur vos propres modèles
4. Consulter la documentation complète

### Ressources:
- Documentation: https://xplia.readthedocs.io
- GitHub: https://github.com/nicolasseverino/xplia
- PyPI: https://pypi.org/project/xplia/